In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install --upgrade pandas


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 28.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 341.8/341.8 kB 25.7 MB/s eta 0:00:00
  Attempting uninstall: pandas
    Found existing installation: pandas 1.5.3
    Uninstalling pandas-1.5.3:
      Successfully uninstalled pandas-1.5.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires pandas==1.5.3, but you have pandas 2.0.3 which is incompatible.


In [3]:
import pandas as pd
import numpy as np
import torch
from tqdm.auto import tqdm
import random
import os

def reset_seeds(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

DATA_PATH = "/content/drive/MyDrive/멀티캠퍼스 자료/Machine Learning/data/"
SEED = 42

device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cpu'

In [4]:
train_nj = pd.read_csv(f"{DATA_PATH}뉴진스_블로그_3개월.csv", encoding='cp949')

In [5]:
train_bp = pd.read_csv(f"{DATA_PATH}블랙핑크_블로그_3개월.csv", encoding='utf-8')

In [6]:
train_ls = pd.read_csv(f"{DATA_PATH}르세라핌_블로그_3개월.csv", encoding='utf-8')

## 크롤링 결과를 하나의 텍스트화 (title + content)

In [7]:
train_nj

,Unnamed: 0,title,datetime,content
0,0,강남역 헤어샵 차홍룸 강남대로점 : 매직으로 뉴진스 머리해보기!,17시간 전,"강남역 헤어샵 차홍룸 강남대로점 :\n반곱슬 머리를 매직으로, 뉴진스 머리해보기! ..."
1,1,버니즈일상?? 뉴진스 1주년 카페투어기,14시간 전,지난주 뉴진스 1주년을 맞이해 1주년 기념 카페 투어를 다녀왔어용~~ 22일이 1주...
2,2,뉴진스 미니 2집 'Get Up' 앨범 도착! 개봉 후기,2023. 7. 24. 4:40,"7월 21일, 한창 대외활동하던 중에 뉴진스 2집이 발매되었고\n바로 다음 날! 구..."
3,3,뉴진스 새 EP 겟업 파워퍼프걸 A버전 개봉 후기!!!,2023. 7. 25. 0:19,#뉴진스 #newjeans #겟업 #getup #뉴진스앨범 #뉴진스겟업 #파워퍼프걸...
4,4,"뉴진스 2집 NJ Box 버전 앨범, 뉴진스 x 파워퍼프걸 알라딘 뉴진스2집,뉴진스...",10시간 전,"저희집 뉴진스팬이 있네요 :) 당연히 저는 아니구요, 저희 아이가 참 좋아해요.\n..."
...,...,...,...,...
143,145,뉴진스 토끼티 & 포토카드 & 앨범,2023. 6. 2. 10:49,"기밀이라고 해서, 거참 어디가서 말할 수는 없지만,\n뉴진스 제작핵심멤버인 R이 나..."
144,146,오렌즈 프렌츠샤인 원데이 웜톤 & 쿨톤을 위한 뉴진스 렌즈,2023. 7. 3. 8:00,오렌즈에서 프렌츠샤인이라는\n신제품을 출시했는데요!\n일명 뉴진스 렌즈!\n뉴진스 ...
145,147,뉴진스 파워퍼프걸 캐릭터 만들기 테스트 링크 mbti 의외로 ENFJ,2023. 6. 25. 23:48,미국의 키치한 감성을 담고 있는\n애니메이션 하면 어떤 애니가\n떠오르시는지 모르겠...
146,148,뉴진스의 사인 축구 셔츠를 받을 수 있는 이벤트 공개 | with 카포스토어,2023. 7. 7. 13:55,대한민국의 자랑스러운 여자 축구 국가대표팀이 2023 FIFA 호주/뉴질랜드 여자 ...


- 뉴진스

In [8]:
train_nj['title'] = train_nj['title'].str.replace("[^a-zA-Z가-힣ㄱ-ㅎ0-9.,!? ]" , "",regex=True)
train_nj['content'] = train_nj['content'].str.replace("[^a-zA-Z가-힣ㄱ-ㅎ0-9.,!? ]" , "",regex=True)

# 제목, 본문내용 합치기
nj_text = train_nj['title'] + train_nj['content']
nj_text

0      강남역 헤어샵 차홍룸 강남대로점  매직으로 뉴진스 머리해보기!강남역 헤어샵 차홍룸 ...
1      버니즈일상?? 뉴진스 1주년 카페투어기지난주 뉴진스 1주년을 맞이해 1주년 기념 카...
2      뉴진스 미니 2집 Get Up 앨범 도착! 개봉 후기7월 21일, 한창 대외활동하던...
3      뉴진스 새 EP 겟업 파워퍼프걸 A버전 개봉 후기!!!뉴진스 newjeans 겟업 ...
4      뉴진스 2집 NJ Box 버전 앨범, 뉴진스 x 파워퍼프걸 알라딘 뉴진스2집,뉴진스...
                             ...                        
143    뉴진스 토끼티  포토카드  앨범기밀이라고 해서, 거참 어디가서 말할 수는 없지만,뉴...
144    오렌즈 프렌츠샤인 원데이 웜톤  쿨톤을 위한 뉴진스 렌즈오렌즈에서 프렌츠샤인이라는신...
145    뉴진스 파워퍼프걸 캐릭터 만들기 테스트 링크 mbti 의외로 ENFJ미국의 키치한 ...
146    뉴진스의 사인 축구 셔츠를 받을 수 있는 이벤트 공개  with 카포스토어대한민국의...
147    뉴진스 파워퍼프걸 콜라보 케이스티파이 powerpuff yourself파워퍼프걸 뉴...
Length: 148, dtype: object

- 블랙핑크

In [9]:
train_bp['title'] = train_bp['title'].str.replace("[^a-zA-Z가-힣ㄱ-ㅎ0-9.,!? ]" , "",regex=True)
train_bp['content'] = train_bp['content'].str.replace("[^a-zA-Z가-힣ㄱ-ㅎ0-9.,!? ]" , "",regex=True)

# 제목, 본문내용 합치기
bp_text = train_bp['title'] + train_bp['content']
bp_text

0      스타벅스 블랙핑크 텀블러 실물 MD 토트백 온라인 스트로베리 초코 크림 프라푸치노,...
1      블랙핑크 로제 설화수 퍼펙팅쿠션! 커버력 좋은 여름 쿠션팩트 추천여름에는 그 어느 ...
2      베트남 블랙핑크 티켓 투기로, 6천만 동 손실 호소블랙핑크 티켓 투기로, 6천만 동...
3      스타벅스 블랙핑크 스트로베리 초코 크림 프라푸치노  MD 텀블러스타벅스 블랙핑크 스...
4      투스젬 뜻, 가격, 부작용? 블랙핑크 리사, 영지 등 연예인들안녕하세요 여러분수여니...
                             ...                        
175    스타벅스 신메뉴 블랙핑크 콜라보 제품 후기 디저트 먹고 꿀팁스벅앱을 들어가니 뜬 소...
176    블랙핑크 제니 열애설 상기 시그널? 방탄소년단BTS 뷔와 파리 데이트룩 자발적 공개...
177    연예인향수 궁금하지 미스디올 블랙핑크 지수 강민경 엄정화 손석구 향수안녕하세요 다양...
178    스벅신상 블랙핑크 딸기 크림 초코번 맛있네제가 좋아하는 두 조합!!블랙핑크  스타벅...
179    베트남 하노이 블랙핑크 환영2층버스 팬들로 가득베트남 하노이 블랙핑크 환영2층버스 ...
Length: 180, dtype: object

- 르세라핌

In [10]:
train_ls['title'] = train_ls['title'].str.replace("[^a-zA-Z가-힣ㄱ-ㅎ0-9.,!? ]" , "",regex=True)
train_ls['content'] = train_ls['content'].str.replace("[^a-zA-Z가-힣ㄱ-ㅎ0-9.,!? ]" , "",regex=True)

# 제목, 본문내용 합치기
ls_text = train_ls['title'] + train_ls['content']
ls_text

0       댄스가수 유랑단, 평창 공연 게스트 르세라핌, 키디비, 퀸와사비, 마마무 별이 빛...
1      허윤진  르세라핌 공식 팬송 피어나  안타깝게도, 감동적인 글 아님... 1주년 팬...
2      르세라핌 노래 이브 프시케 그리고 푸른수염 뜻 정리! 푸른수염의 아내안녕하세요, 쪼...
3      성수 데이트 르세라핌 팝업  향수 팝업 후기요즘 핫한 성수에 르세라핌 팝업이 있다길...
4      댄스가수유랑단 양양 숙소 원두막 평창 무대 촬영지 엄정화 르세라핌 공연 보아 고글 ...
                             ...                        
173    230521 우지세븐틴, 리즈아이브, 채원르세라핌, 있지채령, 예지, 시은스테이씨,...
174    공연정보 르세라핌 서울콘서트2023 LE SSERAFIM TOUR FLAME RIS...
175    르세라핌 일본 방송 음악의 날 출연 일본반응httpsyoutu.beWAnC8IJo4...
176    르세라핌 카즈하 국적 프로필 사진 당신이 모르는 이야기여자아이돌 전성시대인 요즘!오...
177    르세라핌 콘서트 투어 티켓팅 2023 LE SSERAFIM TOUR FLAME RI...
Length: 178, dtype: object

## Kiwi로 토큰화 및 형용사 추출

In [11]:
!pip install kiwipiepy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.5/30.5 MB 24.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for kiwipiepy-model: filename=kiwipiepy_model-0.15.0-py3-none-any.whl size=30602629 sha256=7e40d715108b654c3adddc0a2e553f4dd54ea7bd8433353ed6edd3a706ffe3ae
  Stored in directory: /root/.cache/pip/wheels/f3/55/41/ca474338ece1bc4314b01445f64ff002d71e19df45575a16e2
Successfully built kiwipiepy-model


In [12]:
from kiwipiepy import Kiwi
from kiwipiepy.utils import Stopwords
stopwords =  Stopwords()
kiwi = Kiwi()

- 뉴진스

In [105]:
nj_va_keyword = []

for idx, text in enumerate(nj_text) :
    result = kiwi.tokenize(text,stopwords=stopwords)
    tmp = [token.form for token in result if token.tag in ['XR']] #형용사
    for idx, word in enumerate(tmp) :
        nj_va_keyword.append(word)


In [108]:
len(set(nj_va_keyword))

163

In [109]:
nj_va_keyword

['시원',
 '깔끔',
 '차분',
 '간단',
 '섬세',
 '어색',
 '적당',
 '솔직',
 '맞이',
 '차분',
 '깨끗',
 '난해',
 '비슷',
 '다양',
 '튼튼',
 '관대',
 '이러',
 '불구',
 '복잡',
 '마땅',
 '웬만',
 '난해',
 '뿌듯',
 '매끈',
 '차분',
 '차분',
 '깔끔',
 '이러',
 '확실',
 '한가',
 '비슷',
 '촉촉',
 '궁금',
 '어떡',
 '청량',
 '시원',
 '똘똘',
 '듬직',
 '엉뚱',
 '아련',
 '자세',
 '궁금',
 '신선',
 '강',
 '아른',
 '애매',
 '애매',
 '애매',
 '그러',
 '잡다',
 '만만',
 '진지',
 '정당',
 '빡빡',
 '빡빡',
 '아련',
 '취약',
 '흥얼',
 '궁금',
 '간략',
 '맞이',
 '듬직',
 '큼직',
 '단단',
 '무난',
 '납작',
 '무난',
 '편',
 '뿌듯',
 '복잡',
 '은은',
 '깔끔',
 '이러',
 '어중간',
 '지저분',
 '이러',
 '궁금',
 '강력',
 '뚜렷',
 '심각',
 '자세',
 '술렁',
 '자세',
 '청초',
 '다채',
 '선명',
 '청량',
 '상큼',
 '당연',
 '청량',
 '간결',
 '아련',
 '솔깃',
 '화려',
 '맞이',
 '성급',
 '탄탄',
 '특별',
 '느긋',
 '애매',
 '모호',
 '느긋',
 '애매',
 '웬만',
 '야비',
 '묵직',
 '관대',
 '신기',
 '신선',
 '확고',
 '다양',
 '절절',
 '평범',
 '평범',
 '한심',
 '유약',
 '무난',
 '풍성',
 '명확',
 '확실',
 '풍성',
 '깔끔',
 '이러',
 '간단',
 '편',
 '간단',
 '신선',
 '간단',
 '신기',
 '용이',
 '불구',
 '과감',
 '대단',
 '느끼',
 '희미',
 '달콤',
 '맞이',
 '찝찝',
 '시원',
 '쾌적',
 '청량',
 '청량',
 '은은',


- 블랙핑크


In [15]:
bp_text = [text for text in bp_text if isinstance(text, str)]
#float 값 때문에 오류나는 것 제거

In [16]:
bp_va_keyword = []

for idx, text in enumerate(bp_text) :
    result = kiwi.tokenize(text,stopwords=stopwords)
    tmp = [token.form for token in result if token.tag in ['VA']] #형용사


    for idx, word in enumerate(tmp) :
        bp_va_keyword.append(word)

In [17]:
tmp = [token.form for token in result if token.tag in ['VA']]

In [18]:
meaning_words = ['많', '크', '좋','새롭','다르','어렵','쉽','작','길','아름답','어리','힘들','젊','아프','빠르','나쁘','싫','무섭','짧',
                 '커다랗','엄청나','지나치','희','검','어둡','옳','괜찮','밝','바쁘','가볍','뜨겁','하얗','예쁘','무겁','맑','즐겁','부드럽',
                 '재미있','약하','낯설','편하','낫','비싸','강하','슬프','춥','놀랍','곱','뛰어나','낡','거칠','안타깝','부끄럽','기쁘','바르',
                 '두렵','반갑','맛있','흔하','착하','차갑','싸','조그맣','외롭','올바르','빨갛','멋지','날카롭','급하','친하','굵','굳','아쉽',
                 '느리','환하','더럽','그립','괴롭','귀엽','우습','가늘','어리석','귀하','둥글','부럽','노랗','귀찮','재밌','세','까맣','두껍',
                 '어지럽','거세','차','흐리','파랗','밉','아깝','시끄럽','눈부시','얇','진하','힘겹','점잖','고프','까다롭','힘차','폭넓',
                 '못하','잘나','사납','두드러지','끈질기','험하', '이쁘',' 누렇',' 시커멓','오래','낯익','배고프','끊임없','남다르','지겹',
                 '비','쓸데없','못나','벅차','묘하','기막히','서럽','서글프','세차','값싸','그르','모질','가엾','잘','참되','허옇','엷',
                 '색다르','동그랗','게으르','메마르','벌겋','쑥스럽','엄하','어이없','가느다랗','너그럽','서투르','하찮','시리','매끄럽',
                 '우스꽝스럽','고달프','서툴','뻔하','값지','멋있','잘생기','질기','멍하','새까맣','연하','추하','틀리','억세','안되','여리',
                 '싱겁','쓰라리','어처구니없','역겹','재미없','수줍','뿌옇','천하','징그럽','순하','흉하','안쓰럽','야하','빼어나','속상하',
                 '유별나','매섭','보잘것없','못생기','못되','시퍼렇','기다랗','짓궂','언짢','잽싸','형편없','값비싸','흥겹','새파랗','둥그렇',
                 '둔하','부시','재빠르','딱하','배부르','싱그럽','별나','쓰리','야무지','새하얗','어질','한결같','활기차','궂','아리','갸날프',
                 '건방지','독하','선하','훤하','시뻘겋','탁하','무디','우렁차','능하','철없','뽀얗','번거롭','해롭','쓸모없','성가시','정겹',
                 '애처롭','퍼렇','분하','관계없','네모나','얄밉','차디차','장하','이롭','달갑','헤프','앳되','후하','푸르르','쎄','힘세',
                 '힘없','보드랍','새빨갛','길다랗','아니꼽','미심쩍','해맑','미덥','수상쩍','굳세','만만찮','동떨어지','껄끄럽','험상궂',
                 '변함없','드세','앙증맞','깡마르','구슬프','날쌔','실없','쓰디쓰','시꺼멓','을씨년스럽','검푸르','발갛','시원찮','뻘겋',
                 '어여쁘','당차','고되','모나','다부지','흐드러지','드럽','짙푸르','약삭빠르','방정맞','뜸하','사이좋','천연덕스럽','무르',
                 '볼품없','상스럽','새카맣','격하','거침없','허하','희뿌옇','앙칼지','심술궂','아리땁','끈덕지','역하','가녀리','노오랗',
                 '굼뜨','희디희','가엽','살갑','하잘것없','정신없','버릇없','올곧','악하','청승맞','별스럽','구리','샛노랗','길하','빈틈없',
                 '필요없','남부럽','멋들어지','쪼그맣','똥그랗','매몰차','파아랗','속좁','막되','노하','얌전하','얇디얇','얍삽하','연푸르',
                 '번거럽','똘똘하','야물딱지','예쁘장하','투박스럽','게을러빠지','껄렁껄렁하','믿음직스럽','사근사근하','기','될성부르',
                 '약아빠지','예스럽','쇼킹하','놀라웁','맑디맑','능청맞','곱디곱','징하','모지','꽁하','따스','늙','유머러스하','예쁘장스럽',
                 '촌시럽','보오얗','동그렇','모자르','모잘르','간지','힘쎄','자애롭','시덥잖','쌍스럽','쑥쓰럽','앙징맞','밉살맞','후덥',
                 '여물','찐하','찡하','둥굴','기차','자애롭','시덥잖','쌍스럽','쑥쓰럽','앙징맞','얄따랗','밉살맞','버얼겋','높푸르','찐하',
                 '찡하','기차','주제넘','꺼멓','어리숙하','얄궂','참하','끄떡없','희망차','기운차','희부옇','안스럽','어줍잖','의롭','능글맞',
                 '간드러지','쌀쌀맞','자그맣','볼썽사납','희멀겋','도드라지','재수없',
 '걸','빠알갛','뜬금없','살지','악착같','멀겋','의좋','꾸밈없','같잖','잡스럽','냉하','싸하','빡세','궁상맞','밉살스럽','하이얗','익살맞','쬐그맣','암팡지','어쭙잖','분별없',
 '달갑잖','똑바르','실하','박하','고깝','특별나','새삼스럽','스스럼없','좀스럽','손색없','후지','짠하','시답잖','허물없','변변찮','따습','맹하','애닯','주책없','뚱하','의심쩍',
 '쬐끄맣','섹시하','곱다랗','시건방지','특출나','옹골차','막돼먹','곰살궂','가소롭','유하','속없','지각없','거멓','대단찮','기똥차','깊푸르','따시','뜨시','맥없','싯누렇','로맨틱하'
 '도탑','허여멀겋','모지락스럽','향그럽','옹골지','성마르','성스럽','꼴사납','시답','동글','마르','대차','조그만하','인상깊','야멸차','염치없','쌩뚱맞','매스껍','극성맞','깊디깊',
 '감명깊','반하','궁하','덜떨어지','나즈막하','요상하','작디작','째쨰하','촌스럽','뽀오얗','색스럽','다사롭','둔덕지','맵싸하','까아맣','감미롭','짖궂','시덥','어줍','몽글',
 '값있','눈꼴사납']

- 르세라핌

In [19]:
ls_va_keyword = []

for idx, text in enumerate(ls_text) :
    result = kiwi.tokenize(text,stopwords=stopwords)
    tmp = [token.form for token in result if token.tag in ['VA']] #형용사


    for idx, word in enumerate(tmp) :
        ls_va_keyword.append(word)

- 형용사 개수 (1500~2500개)

In [20]:
len(nj_va_keyword), len(bp_va_keyword), len(ls_va_keyword)

(1855, 1562, 2467)

## 불용어 및 동의어 처리
- 예쁨-이쁨, 재미있음-재밌음

- 불용어 처리

In [21]:
stopword_list = ['어떻','이렇','그렇','저렇','요렇','있','이러하','저러하', #의미없음
                 '많','적','길','수많','커다랗','각지','자세하','같','비슷하', #의미 있으나 사람 표현 아님
                 '답답하','귀찮','힘들', #의미 있으나 인물에 대한 반응 아님
                 '좋','싫','이쁘','재미있'] #중복 (이쁘-예쁘, 재미있-재밌) or 너무 넓은 의미 (좋, 싫)

nj_nva_list = [x for x in nj_va_keyword if x not in stopword_list]
bp_nva_list = [x for x in bp_va_keyword if x not in stopword_list]
ls_nva_list = [x for x in ls_va_keyword if x not in stopword_list]

## 형용사를 명사화

- 형용사를 명사화하는 함수 적용 (adjective_to_noun)
    - 마지막 글자에 받침이 있으면 '음'을, 없으면 받침에 ㅁ을 더함
    

In [22]:
def adjective_to_noun(adjective_list):
    results = []
    for adjective in adjective_list:
        last_char = adjective[-1]

        if (ord(last_char) - 44032) % 28 != 0: #받침이 있다면 음을 더하고
            result = adjective + "음"
        else:
            last_jamo = chr(ord(last_char) + 16) #없다면 마지막 글자에 ㅁ 받침을 추가
            result = adjective[:-1] + last_jamo

        results.append(result)
    return results

In [23]:
nj_nva_list = adjective_to_noun(nj_nva_list)
bp_nva_list = adjective_to_noun(bp_nva_list)
ls_nva_list = adjective_to_noun(ls_nva_list)

In [24]:
ls_nva_list

['멋짐',
 '멋짐',
 '핫함',
 '맛있음',
 '맛있음',
 '맛있음',
 '예쁨',
 '셈',
 '재밌음',
 '재밌음',
 '멋짐',
 '색다름',
 '멋짐',
 '멋짐',
 '멋짐',
 '멋짐',
 '멋짐',
 '시림',
 '큼',
 '큼',
 '큼',
 '큼',
 '하찮음',
 '큼',
 '멀쩡함',
 '심각함',
 '멋짐',
 '밝음',
 '고픔',
 '다름',
 '멋짐',
 '똑같음',
 '재밌음',
 '다름',
 '멋짐',
 '푸름',
 '멋짐',
 '멋짐',
 '멋있음',
 '변함없음',
 '푸름',
 '푸름',
 '푸름',
 '궁금함',
 '구림',
 '궁금함',
 '푸름',
 '푸름',
 '푸름',
 '괜찮음',
 '핫함',
 '흐림',
 '흐림',
 '충분함',
 '맛있음',
 '강함',
 '재밌음',
 '시원함',
 '맛있음',
 '맛있음',
 '맛있음',
 '재밌음',
 '재밌음',
 '시원함',
 '아늑함',
 '저렴함',
 '화려함',
 '큼',
 '깊음',
 '뜻깊음',
 '훈훈함',
 '멋짐',
 '멋짐',
 '심함',
 '심함',
 '기쁨',
 '뿌듯함',
 '멋짐',
 '큼',
 '깊음',
 '예쁨',
 '푸름',
 '높음',
 '강함',
 '빠름',
 '셈',
 '높음',
 '멋짐',
 '푸름',
 '푸름',
 '큼',
 '바쁨',
 '큼',
 '젊음',
 '큼',
 '비쌈',
 '저렴함',
 '큼',
 '다름',
 '늦음',
 '쌈',
 '밝음',
 '괜찮음',
 '빠름',
 '빠름',
 '뛰어남',
 '뛰어남',
 '강함',
 '바름',
 '소용없음',
 '뛰어남',
 '멋짐',
 '괜찮음',
 '조악함',
 '무색함',
 '맛있음',
 '상큼함',
 '맛있음',
 '큼',
 '큼',
 '솔직함',
 '나쁨',
 '바름',
 '나쁨',
 '빠름',
 '친함',
 '깊음',
 '뽀얗음',
 '밝음',
 '나쁨',
 '흐릿함',
 '밝음',
 '밝음',
 '밝음',
 '나쁨',
 '탁함',
 '탁함',
 '탁함',
 

## TF-IDF 기반 top 20 키워드 선정

- TF-IDF 함수 정의

In [25]:
!pip install konlpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 30.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 465.3/465.3 kB 26.5 MB/s eta 0:00:00


In [26]:
from konlpy.tag import Kkma
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np

In [27]:
def extract_keywords_tfidf(text, num_keywords=5):
    kkma = Kkma()
    # nouns = kkma.nouns(text)
    # preprocessed_text = " ".join(nouns)

    tfidf_vectorizer = TfidfVectorizer()
    tfidf_matrix = tfidf_vectorizer.fit_transform([text])
    feature_names = tfidf_vectorizer.get_feature_names_out()

    # TF-IDF 값에 따라 정렬된 단어들의 인덱스를 얻음
    sorted_indices = tfidf_matrix.toarray()[0].argsort()[::-1]

   # 상위 num_keywords 개의 단어와 해당 단어들의 TF-IDF 스코어 추출
    top_keywords = [(feature_names[idx], tfidf_matrix[0, idx]) for idx in sorted_indices[:num_keywords]]

    return top_keywords

- 세 텍스트에 대해 TF-IDF 키워드 추출

In [28]:
nj_nva_string = ', '.join(nj_nva_list)
bp_nva_string = ', '.join(bp_nva_list)
ls_nva_string = ', '.join(ls_nva_list)

- 뉴진스

In [29]:
text = nj_nva_string

top_keywords_tfidf = extract_keywords_tfidf(text,num_keywords=20)

nj_tfidf_top20 = []

print("TF-IDF 기반 키워드 및 스코어 추출:")
for keyword, score in top_keywords_tfidf:
    nj_tfidf_top20.append(keyword)
    print(f"{keyword} - Score: {score}")

TF-IDF 기반 키워드 및 스코어 추출:
예쁨 - Score: 0.8187023585462245
다름 - Score: 0.3047724108456748
맑음 - Score: 0.22708532572814988
재밌음 - Score: 0.1852538183571749
맛있음 - Score: 0.1493982406106249
작음 - Score: 0.11354266286407494
괜찮음 - Score: 0.10159080361522493
핫함 - Score: 0.09561487399079995
높음 - Score: 0.08963894436637494
궁금함 - Score: 0.08366301474194995
멋짐 - Score: 0.08366301474194995
짧음 - Score: 0.07768708511752495
뛰어남 - Score: 0.06573522586867496
밝음 - Score: 0.06573522586867496
강함 - Score: 0.06573522586867496
어림 - Score: 0.06573522586867496
비쌈 - Score: 0.059759296244249964
빠름 - Score: 0.05378336661982497
엄청남 - Score: 0.05378336661982497
검음 - Score: 0.05378336661982497


- 블랙핑크

In [30]:
text = bp_nva_string

top_keywords_tfidf = extract_keywords_tfidf(text,num_keywords=20)

bp_tfidf_top20 = []

print("TF-IDF 기반 키워드 및 스코어 추출:")
for keyword, score in top_keywords_tfidf:
    bp_tfidf_top20.append(keyword)
    print(f"{keyword} - Score: {score}")

TF-IDF 기반 키워드 및 스코어 추출:
예쁨 - Score: 0.779798272190714
맛있음 - Score: 0.3351764503275876
높음 - Score: 0.23257141451301994
다름 - Score: 0.19836973590816406
얇음 - Score: 0.18468906446622171
핫함 - Score: 0.17784872874525054
비쌈 - Score: 0.1094453715355388
작음 - Score: 0.10260503581456762
달달함 - Score: 0.10260503581456762
밝음 - Score: 0.08892436437262527
촉촉함 - Score: 0.08892436437262527
진함 - Score: 0.0820840286516541
괜찮음 - Score: 0.07524369293068292
엄청남 - Score: 0.07524369293068292
편함 - Score: 0.06840335720971175
궁금함 - Score: 0.06840335720971175
빠름 - Score: 0.06156302148874057
멋짐 - Score: 0.0547226857677694
넓음 - Score: 0.04788235004679822
강함 - Score: 0.04788235004679822


- 르세라핌

In [31]:
text = ls_nva_string

top_keywords_tfidf = extract_keywords_tfidf(text,num_keywords=20)

ls_tfidf_top20 = []

print("TF-IDF 기반 키워드 및 스코어 추출:")
for keyword, score in top_keywords_tfidf:
    ls_tfidf_top20.append(keyword)
    print(f"{keyword} - Score: {score}")

TF-IDF 기반 키워드 및 스코어 추출:
예쁨 - Score: 0.5200858741757377
푸름 - Score: 0.4279187572332019
다름 - Score: 0.3291682747947707
멋짐 - Score: 0.2962514473152936
맛있음 - Score: 0.23041779235633947
높음 - Score: 0.1975009648768624
뛰어남 - Score: 0.15800077190148992
강함 - Score: 0.1448340409096991
빠름 - Score: 0.13825067541380368
재밌음 - Score: 0.13825067541380368
짧음 - Score: 0.12508394442201284
핫함 - Score: 0.12508394442201284
괜찮음 - Score: 0.11850057892611744
작음 - Score: 0.0987504824384312
깊음 - Score: 0.0987504824384312
밝음 - Score: 0.09216711694253579
어림 - Score: 0.09216711694253579
엄청남 - Score: 0.08558375144664038
궁금함 - Score: 0.07900038595074496
나쁨 - Score: 0.07900038595074496


In [32]:
df_nj = pd.DataFrame(nj_tfidf_top20)
df_bp = pd.DataFrame(bp_tfidf_top20)
df_ls = pd.DataFrame(ls_tfidf_top20)

df_nj.columns = ['뉴진스']
df_bp.columns = ['블랙핑크']
df_ls.columns = ['르세라핌']

pd.concat([df_nj, df_bp, df_ls],axis=1)

,뉴진스,블랙핑크,르세라핌
0,예쁨,예쁨,예쁨
1,다름,맛있음,푸름
2,맑음,높음,다름
3,재밌음,다름,멋짐
4,맛있음,얇음,맛있음
5,작음,핫함,높음
6,괜찮음,비쌈,뛰어남
7,핫함,작음,강함
8,높음,달달함,빠름
9,궁금함,밝음,재밌음


## 키워드 분석 및 비교

### 공통 및 개별 키워드

- 뉴진스, 블랙핑크, 르세라핌 공통 키워드 : Top20




In [33]:
# 각 리스트를 집합(set)으로 변환
nj_set_top20 = set(nj_tfidf_top20)
bp_set_top20 = set(bp_tfidf_top20)
ls_set_top20 = set(ls_tfidf_top20)

# 공통 키워드: 모든 리스트에 있는 키워드
common_keywords = nj_set_top20.intersection(bp_set_top20, ls_set_top20)

print("공통 키워드:", common_keywords)
print('공통 키워드 개수:', len(common_keywords)) #50% 정도 겹침

공통 키워드: {'멋짐', '엄청남', '궁금함', '강함', '예쁨', '다름', '높음', '밝음', '맛있음', '작음', '빠름', '핫함', '괜찮음'}
공통 키워드 개수: 13


In [34]:
# common list에 있는 키워드 제외
nj_unique_top20 = nj_set_top20.difference(common_keywords)
bp_unique_top20 = bp_set_top20.difference(common_keywords)
ls_unique_top20 = ls_set_top20.difference(common_keywords)

print("뉴진스 키워드:", nj_unique_top20)
print("블랙핑크 키워드:", bp_unique_top20)
print("르세라핌 키워드:", ls_unique_top20)

뉴진스 키워드: {'재밌음', '짧음', '어림', '비쌈', '맑음', '뛰어남', '검음'}
블랙핑크 키워드: {'얇음', '넓음', '달달함', '비쌈', '진함', '편함', '촉촉함'}
르세라핌 키워드: {'재밌음', '짧음', '어림', '푸름', '깊음', '뛰어남', '나쁨'}


In [35]:
# 각자 서로 다른 키워드: 각 리스트에만 있는 키워드
nj_unique_top20 = nj_set_top20.difference(bp_set_top20, ls_set_top20)
bp_unique_top20 = bp_set_top20.difference(nj_set_top20, ls_set_top20)
ls_unique_top20 = ls_set_top20.difference(nj_set_top20, bp_set_top20)

print("뉴진스 키워드:", nj_unique_top20)
print("블랙핑크 키워드:", bp_unique_top20)
print("르세라핌 키워드:", ls_unique_top20)

뉴진스 키워드: {'맑음', '검음'}
블랙핑크 키워드: {'얇음', '넓음', '달달함', '진함', '편함', '촉촉함'}
르세라핌 키워드: {'푸름', '깊음', '나쁨'}


- 뉴진스, 블랙핑크, 르세라핌 공통 키워드 : Top50


In [36]:
text = nj_nva_string
top_keywords_tfidf = extract_keywords_tfidf(text,num_keywords=50)

nj_tfidf_top50 = []
for keyword, score in top_keywords_tfidf:
    nj_tfidf_top50.append(keyword)

In [37]:
text = bp_nva_string
top_keywords_tfidf = extract_keywords_tfidf(text,num_keywords=50)

bp_tfidf_top50 = []
for keyword, score in top_keywords_tfidf:
    bp_tfidf_top50.append(keyword)

In [38]:
text = ls_nva_string
top_keywords_tfidf = extract_keywords_tfidf(text,num_keywords=50)

ls_tfidf_top50 = []
for keyword, score in top_keywords_tfidf:
    ls_tfidf_top50.append(keyword)

In [39]:
# 각 리스트를 집합(set)으로 변환
nj_set_top50 = set(nj_tfidf_top50)
bp_set_top50 = set(bp_tfidf_top50)
ls_set_top50 = set(ls_tfidf_top50)

# 공통 키워드: 모든 리스트에 있는 키워드
common_keywords = nj_set_top50.intersection(bp_set_top50, ls_set_top50)

print("공통 키워드:", common_keywords)
print('공통 키워드 개수:', len(common_keywords)) #50% 정도 겹침

공통 키워드: {'짧음', '강함', '나쁨', '괜찮음', '과함', '비쌈', '진함', '예쁨', '다름', '넓음', '쿨함', '밝음', '높음', '맛있음', '작음', '빠름', '멋짐', '엄청남', '궁금함', '똑같음', '늦음', '뛰어남', '편함', '핫함'}
공통 키워드 개수: 24


In [40]:
# common list에 있는 키워드 제외
nj_unique_top50 = nj_set_top50.difference(common_keywords)
bp_unique_top50 = bp_set_top50.difference(common_keywords)
ls_unique_top50 = ls_set_top50.difference(common_keywords)

print("뉴진스 키워드:", nj_unique_top50)
print("블랙핑크 키워드:", bp_unique_top50)
print("르세라핌 키워드:", ls_unique_top50)

뉴진스 키워드: {'눈부심', '또렷함', '바쁨', '달달함', '깊음', '노랗음', '어림', '은은함', '무난함', '잘음', '멀음', '기쁨', '흐림', '재밌음', '대단함', '멋있음', '맑음', '알참', '낮음', '검음', '색다름', '상관없음', '착함', '깜찍함', '아픔', '여림'}
블랙핑크 키워드: {'신기함', '빨갛음', '얇음', '섬세함', '바쁨', '달달함', '촘촘함', '슬픔', '노랗음', '바름', '습함', '은은함', '꾸준함', '촉촉함', '고름', '별다름', '낮음', '향긋함', '착함', '아픔', '달음', '로맨틱함', '우아함', '시원함', '남다름', '짙음'}
르세라핌 키워드: {'눈부심', '얇음', '푸름', '깊음', '낯설음', '어림', '낡음', '저렴함', '기쁨', '젊음', '흐림', '재밌음', '뿌듯함', '대단함', '멋있음', '탁함', '맑음', '하얗음', '색다름', '상관없음', '약함', '힘참', '상큼함', '우아함', '시원함', '남다름'}


In [41]:
# 각자 서로 다른 키워드: 각 리스트에만 있는 키워드
nj_unique_top50 = nj_set_top50.difference(bp_set_top50, ls_set_top50)
bp_unique_top50 = bp_set_top50.difference(nj_set_top50, ls_set_top50)
ls_unique_top50 = ls_set_top50.difference(nj_set_top50, bp_set_top50)

print("뉴진스 키워드:", nj_unique_top50)
print("블랙핑크 키워드:", bp_unique_top50)
print("르세라핌 키워드:", ls_unique_top50)

뉴진스 키워드: {'또렷함', '무난함', '잘음', '멀음', '알참', '검음', '깜찍함', '여림'}
블랙핑크 키워드: {'신기함', '빨갛음', '섬세함', '촘촘함', '슬픔', '바름', '습함', '꾸준함', '촉촉함', '고름', '별다름', '향긋함', '달음', '로맨틱함', '짙음'}
르세라핌 키워드: {'약함', '뿌듯함', '힘참', '상큼함', '하얗음', '푸름', '탁함', '낡음', '낯설음', '저렴함', '젊음'}


- 뉴진스, 블랙핑크, 르세라핌 공통 키워드 : Top100


In [42]:
text = nj_nva_string
top_keywords_tfidf = extract_keywords_tfidf(text,num_keywords=100)

nj_tfidf_top100 = []
for keyword, score in top_keywords_tfidf:
    nj_tfidf_top100.append(keyword)

In [43]:
text = bp_nva_string
top_keywords_tfidf = extract_keywords_tfidf(text,num_keywords=100)

bp_tfidf_top100 = []
for keyword, score in top_keywords_tfidf:
    bp_tfidf_top100.append(keyword)

In [44]:
text = ls_nva_string
top_keywords_tfidf = extract_keywords_tfidf(text,num_keywords=100)

ls_tfidf_top100 = []
for keyword, score in top_keywords_tfidf:
    ls_tfidf_top100.append(keyword)

In [45]:
# 각 리스트를 집합(set)으로 변환
nj_set_top100 = set(nj_tfidf_top100)
bp_set_top100 = set(bp_tfidf_top100)
ls_set_top100 = set(ls_tfidf_top100)

# 공통 키워드: 모든 리스트에 있는 키워드
common_keywords = nj_set_top100.intersection(bp_set_top100, ls_set_top100)

print("공통 키워드:", common_keywords)
print('공통 키워드 개수:', len(common_keywords)) #50% 정도 겹침

공통 키워드: {'짧음', '강함', '마름', '바쁨', '달달함', '깊음', '나쁨', '괜찮음', '은은함', '무난함', '핫함', '과함', '비쌈', '달콤함', '연함', '진함', '저렴함', '재밌음', '흔함', '대단함', '예쁨', '다름', '넓음', '멋있음', '밝음', '쿨함', '작음', '맛있음', '맑음', '걸맞음', '검음', '빠름', '상관없음', '착함', '멋짐', '엄청남', '궁금함', '똑같음', '시원함', '남다름', '늦음', '뛰어남', '편함', '높음'}
공통 키워드 개수: 44


In [46]:
# common list에 있는 키워드 제외
nj_unique_top100 = nj_set_top100.difference(common_keywords)
bp_unique_top100 = bp_set_top100.difference(common_keywords)
ls_unique_top100 = ls_set_top100.difference(common_keywords)

print("뉴진스 키워드:", nj_unique_top100)
print("블랙핑크 키워드:", bp_unique_top100)
print("르세라핌 키워드:", ls_unique_top100)

뉴진스 키워드: {'대견함', '신기함', '빨갛음', '잦음', '눈부심', '또렷함', '거칠음', '담백함', '따스함', '푸름', '고고함', '매콤함', '낯설음', '까맣음', '평범함', '따끈함', '슬픔', '노랗음', '그러함', '습함', '어림', '드셈', '파랗음', '반함', '잘음', '붉음', '간략함', '심함', '멀음', '기쁨', '젊음', '흐림', '촉촉함', '신선함', '고름', '뿌듯함', '따듯함', '기막힘', '지나침', '알참', '맛남', '낮음', '간편함', '하얗음', '색다름', '빡빡함', '묘함', '냉함', '깜찍함', '아픔', '동그랗음', '따뜻함', '뒤늦음', '달음', '귀함', '여림'}
블랙핑크 키워드: {'굉장함', '굵음', '적당함', '탄탄함', '신기함', '빨갛음', '섬세함', '얇음', '간단함', '매끈함', '잘생김', '고고함', '충분함', '촘촘함', '슬픔', '노랗음', '바름', '습함', '배고픔', '넉넉함', '정교함', '고소함', '훌륭함', '기나길음', '꾸준함', '강렬함', '낯익음', '뜨끈함', '뚜렷함', '촉촉함', '고름', '별다름', '느끼함', '익숙함', '느림', '나른함', '향긋함', '맛남', '낮음', '간편함', '과감함', '급함', '단정함', '묘함', '아픔', '뒤늦음', '다름없음', '달음', '멍함', '퍽퍽함', '로맨틱함', '우아함', '새콤함', '쾌적함', '짙음', '날씬함'}
르세라핌 키워드: {'굵음', '친함', '탄탄함', '잦음', '눈부심', '가늘음', '얇음', '거칠음', '푸름', '훈훈함', '당참', '결연함', '솔직함', '낯설음', '고픔', '활기참', '자그맣음', '바름', '폭넓음', '어림', '형편없음', '옳음', '재미없음', '빡셈', '꾸준함', '드물음', '낡음', '심함', '붉음', '기쁨', '젊음', '흐림', '화려함', '신

In [47]:
# 각자 서로 다른 키워드: 각 리스트에만 있는 키워드
nj_unique_top100 = nj_set_top50.difference(bp_set_top100, ls_set_top100)
bp_unique_top100 = bp_set_top50.difference(nj_set_top100, ls_set_top100)
ls_unique_top100 = ls_set_top50.difference(nj_set_top100, bp_set_top100)

print("뉴진스 키워드:", nj_unique_top100)
print("블랙핑크 키워드:", bp_unique_top100)
print("르세라핌 키워드:", ls_unique_top100)

뉴진스 키워드: {'깜찍함', '또렷함', '잘음', '알참', '멀음', '여림'}
블랙핑크 키워드: {'별다름', '섬세함', '로맨틱함', '향긋함', '짙음', '촘촘함'}
르세라핌 키워드: {'약함', '힘참', '상큼함', '탁함', '낡음'}


- 전체 키워드 리스트 (meaning words에 있는)

In [104]:
meaning_words = ['많', '크', '좋','새롭','다르','어렵','쉽','작','길','아름답','어리','힘들','젊','아프','빠르','나쁘','싫','무섭','짧',
                 '커다랗','엄청나','지나치','희','검','어둡','옳','괜찮','밝','바쁘','가볍','뜨겁','하얗','예쁘','무겁','맑','즐겁','부드럽',
                 '재미있','약하','낯설','편하','낫','비싸','강하','슬프','춥','놀랍','곱','뛰어나','낡','거칠','안타깝','부끄럽','기쁘','바르',
                 '두렵','반갑','맛있','흔하','착하','차갑','싸','조그맣','외롭','올바르','빨갛','멋지','날카롭','급하','친하','굵','굳','아쉽',
                 '느리','환하','더럽','그립','괴롭','귀엽','우습','가늘','어리석','귀하','둥글','부럽','노랗','귀찮','재밌','세','까맣','두껍',
                 '어지럽','거세','차','흐리','파랗','밉','아깝','시끄럽','눈부시','얇','진하','힘겹','점잖','고프','까다롭','힘차','폭넓',
                 '못하','잘나','사납','두드러지','끈질기','험하', '이쁘',' 누렇',' 시커멓','오래','낯익','배고프','끊임없','남다르','지겹',
                 '비','쓸데없','못나','벅차','묘하','기막히','서럽','서글프','세차','값싸','그르','모질','가엾','잘','참되','허옇','엷',
                 '색다르','동그랗','게으르','메마르','벌겋','쑥스럽','엄하','어이없','가느다랗','너그럽','서투르','하찮','시리','매끄럽',
                 '우스꽝스럽','고달프','서툴','뻔하','값지','멋있','잘생기','질기','멍하','새까맣','연하','추하','틀리','억세','안되','여리',
                 '싱겁','쓰라리','어처구니없','역겹','재미없','수줍','뿌옇','천하','징그럽','순하','흉하','안쓰럽','야하','빼어나','속상하',
                 '유별나','매섭','보잘것없','못생기','못되','시퍼렇','기다랗','짓궂','언짢','잽싸','형편없','값비싸','흥겹','새파랗','둥그렇',
                 '둔하','부시','재빠르','딱하','배부르','싱그럽','별나','쓰리','야무지','새하얗','어질','한결같','활기차','궂','아리','갸날프',
                 '건방지','독하','선하','훤하','시뻘겋','탁하','무디','우렁차','능하','철없','뽀얗','번거롭','해롭','쓸모없','성가시','정겹',
                 '애처롭','퍼렇','분하','관계없','네모나','얄밉','차디차','장하','이롭','달갑','헤프','앳되','후하','푸르르','쎄','힘세',
                 '힘없','보드랍','새빨갛','길다랗','아니꼽','미심쩍','해맑','미덥','수상쩍','굳세','만만찮','동떨어지','껄끄럽','험상궂',
                 '변함없','드세','앙증맞','깡마르','구슬프','날쌔','실없','쓰디쓰','시꺼멓','을씨년스럽','검푸르','발갛','시원찮','뻘겋',
                 '어여쁘','당차','고되','모나','다부지','흐드러지','드럽','짙푸르','약삭빠르','방정맞','뜸하','사이좋','천연덕스럽','무르',
                 '볼품없','상스럽','새카맣','격하','거침없','허하','희뿌옇','앙칼지','심술궂','아리땁','끈덕지','역하','가녀리','노오랗',
                 '굼뜨','희디희','가엽','살갑','하잘것없','정신없','버릇없','올곧','악하','청승맞','별스럽','구리','샛노랗','길하','빈틈없',
                 '필요없','남부럽','멋들어지','쪼그맣','똥그랗','매몰차','파아랗','속좁','막되','노하','얌전하','얇디얇','얍삽하','연푸르',
                 '번거럽','똘똘하','야물딱지','예쁘장하','투박스럽','게을러빠지','껄렁껄렁하','믿음직스럽','사근사근하','기','될성부르',
                 '약아빠지','예스럽','쇼킹하','놀라웁','맑디맑','능청맞','곱디곱','징하','모지','꽁하','따스','늙','유머러스하','예쁘장스럽',
                 '촌시럽','보오얗','동그렇','모자르','모잘르','간지','힘쎄','자애롭','시덥잖','쌍스럽','쑥쓰럽','앙징맞','밉살맞','후덥',
                 '여물','찐하','찡하','둥굴','기차','자애롭','시덥잖','쌍스럽','쑥쓰럽','앙징맞','얄따랗','밉살맞','버얼겋','높푸르','찐하',
                 '찡하','기차','주제넘','꺼멓','어리숙하','얄궂','참하','끄떡없','희망차','기운차','희부옇','안스럽','어줍잖','의롭','능글맞',
                 '간드러지','쌀쌀맞','자그맣','볼썽사납','희멀겋','도드라지','재수없','걸','빠알갛','뜬금없','살지','악착같','멀겋','의좋','꾸밈없',
                 '같잖','잡스럽','냉하','싸하','빡세','궁상맞','밉살스럽','하이얗','익살맞','쬐그맣','암팡지','어쭙잖','분별없','달갑잖','똑바르',
                 '실하','박하','고깝','특별나','새삼스럽','스스럼없','좀스럽','손색없','후지','짠하','시답잖','허물없','변변찮','따습','맹하',
                 '애닯','주책없','뚱하','의심쩍','쬐끄맣','섹시하','곱다랗','시건방지','특출나','옹골차','막돼먹','곰살궂','가소롭','유하','속없',
                 '지각없','거멓','대단찮','기똥차','깊푸르','따시','뜨시','맥없','싯누렇','로맨틱하','도탑','허여멀겋','모지락스럽','향그럽',
                 '옹골지','성마르','성스럽','꼴사납','시답','동글','마르','대차','조그만하','인상깊','야멸차','염치없','쌩뚱맞','매스껍','극성맞',
                 '깊디깊','감명깊','반하','궁하','덜떨어지','나즈막하','요상하','작디작','째쨰하','촌스럽','뽀오얗','색스럽','다사롭','둔덕지',
                 '맵싸하','까아맣','감미롭','짖궂','시덥','어줍','몽글','값있','눈꼴사납']

In [85]:
nj_va_keyword = []

for idx, text in enumerate(nj_text) :
    result = kiwi.tokenize(text,stopwords=stopwords)
    tmp = [token.form for token in result if token.tag in ['VA']] #형용사
    for idx, word in enumerate(tmp) :
        nj_va_keyword.append(word)


In [ ]:
nj_va_keyword

In [55]:
nj_va_meaningwords =[]

for idx, text in enumerate(nj_text) :
    result = kiwi.tokenize(text,stopwords=stopwords)
    tmp = [token.form for token in result if token.tag in ['VA'] and token.form in meaning_words] #형용사
    for idx, word in enumerate(tmp) :
        nj_va_meaningwords.append(word)

In [ ]:
nj_va_meaningwords

In [78]:
#3월치 뉴진스 블로그 형용사 리스트

len(set(nj_va_keyword)), len(set(nj_nva_meaningwords)) #약 100개가 걸러짐

(187, 88)

In [86]:
nj_nva_meaningwords = adjective_to_noun(nj_va_meaningwords)
nj_nva_keyword = adjective_to_noun(nj_va_keyword)

In [87]:
nj_nva_str_meaningwords = ', '.join(nj_nva_meaningwords)
nj_nva_str_keyword = ', '.join(nj_va_keyword)


In [97]:
text = nj_nva_str_keyword
top_keywords_tfidf = extract_keywords_tfidf(text,num_keywords=300)

nj_tfidf_top300 = []
for keyword, score in top_keywords_tfidf:
    nj_tfidf_top300.append(keyword)

In [98]:
text = nj_nva_str_meaningwords
top_keywords_tfidf = extract_keywords_tfidf(text,num_keywords=300)

meaningwords_nj_tfidf_top300 = []
for keyword, score in top_keywords_tfidf:
    meaningwords_nj_tfidf_top300.append(keyword)

In [99]:
nj_tfidf_top300

['예쁘',
 '그렇',
 '이렇',
 '다르',
 '재밌',
 '이쁘',
 '어떻',
 '맛있',
 '요렇',
 '괜찮',
 '핫하',
 '재미있',
 '궁금하',
 '비슷하',
 '멋지',
 '강하',
 '뛰어나',
 '어리',
 '비싸',
 '엄청나',
 '빠르',
 '똑같',
 '힘들',
 '멋있',
 '편하',
 '대단하',
 '기쁘',
 '또렷하',
 '바쁘',
 '은은하',
 '쿨하',
 '아프',
 '귀찮',
 '답답하',
 '알차',
 '과하',
 '나쁘',
 '깜찍하',
 '여리',
 '상관없',
 '색다르',
 '노랗',
 '눈부시',
 '흐리',
 '무난하',
 '달달하',
 '진하',
 '저렴하',
 '하얗',
 '간편하',
 '신기하',
 '착하',
 '빨갛',
 '묘하',
 '반하',
 '연하',
 '뿌듯하',
 '흔하',
 '따뜻하',
 '빡빡하',
 '남다르',
 '뒤늦',
 '수많',
 '파랗',
 '슬프',
 '습하',
 '시원하',
 '이러하',
 '신선하',
 '심하',
 '매콤하',
 '대견하',
 '촉촉하',
 '지나치',
 '평범하',
 '담백하',
 '고르',
 '푸르',
 '까맣',
 '거칠',
 '맛나',
 '걸맞',
 '간략하',
 '마르',
 '그러하',
 '따스하',
 '따듯하',
 '따끈하',
 '드세',
 '둥글',
 '동그랗',
 '달콤하',
 '고고하',
 '다름없',
 '능수능란하',
 '냉하',
 '귀하',
 '낯설',
 '깨끗하',
 '기막히',
 '힘차',
 '섬세하',
 '멍하',
 '친하',
 '저렇',
 '정신없',
 '조그마하',
 '좋아하',
 '지저분하',
 '짓궂',
 '짠하',
 '철없',
 '충분하',
 '치밀하',
 '커다랗',
 '자그마하',
 '텁텁하',
 '폭넓',
 '푸르르',
 '푸짐하',
 '풋풋하',
 '화기애애하',
 '화려하',
 '확고하',
 '활기차',
 '흐릿하',
 '재미없',
 '익숙하',
 '목마르',
 '세세하',
 '밋밋하',
 '바르',
 '부

In [100]:
meaningwords_nj_tfidf_top300

['좋음',
 '예쁨',
 '많음',
 '다름',
 '맑음',
 '재밌음',
 '이쁨',
 '맛있음',
 '작음',
 '길음',
 '괜찮음',
 '재미있음',
 '멋짐',
 '짧음',
 '밝음',
 '뛰어남',
 '어림',
 '강함',
 '비쌈',
 '빠름',
 '엄청남',
 '검음',
 '멋있음',
 '힘들음',
 '편함',
 '기쁨',
 '바쁨',
 '아픔',
 '나쁨',
 '귀찮음',
 '여림',
 '진함',
 '색다름',
 '눈부심',
 '흐림',
 '싫음',
 '노랗음',
 '하얗음',
 '묘함',
 '잘음',
 '착함',
 '빨갛음',
 '반함',
 '흔함',
 '지나침',
 '슬픔',
 '파랗음',
 '남다름',
 '젊음',
 '까맣음',
 '연함',
 '둥글음',
 '동그랗음',
 '냉함',
 '드셈',
 '낯설음',
 '거칠음',
 '귀함',
 '마름',
 '기막힘',
 '힘참',
 '멍함',
 '바름',
 '활기참',
 '푸르름',
 '폭넓음',
 '커다랗음',
 '친함',
 '철없음',
 '짠함',
 '짓궂음',
 '정신없음',
 '재미없음',
 '옳음',
 '역함',
 '어리석음',
 '얇음',
 '약함',
 '앙증맞음',
 '쓸데없음',
 '새하얗음',
 '부심',
 '쓰림']

In [101]:
len(nj_tfidf_top300),len(meaningwords_nj_tfidf_top300)

(153, 83)

In [102]:
# 각 리스트를 집합(set)으로 변환
nj_set_top300 = set(nj_tfidf_top300)
meaningwords_nj_set_top300 = set(meaningwords_nj_tfidf_top300)

# 공통 키워드: 모든 리스트에 있는 키워드
common_keywords = nj_set_top100.intersection(meaningwords_nj_tfidf_top300)

print("공통 키워드:", common_keywords)
print('공통 키워드 개수:', len(common_keywords)) #50% 정도 겹침

공통 키워드: {'짧음', '강함', '빨갛음', '마름', '눈부심', '거칠음', '바쁨', '낯설음', '까맣음', '나쁨', '괜찮음', '노랗음', '슬픔', '어림', '드셈', '파랗음', '반함', '비쌈', '잘음', '진함', '연함', '기쁨', '젊음', '흐림', '재밌음', '흔함', '예쁨', '다름', '기막힘', '멋있음', '밝음', '지나침', '맑음', '맛있음', '작음', '검음', '빠름', '하얗음', '색다름', '묘함', '멋짐', '착함', '동그랗음', '아픔', '엄청남', '냉함', '귀함', '남다름', '뛰어남', '편함', '여림'}
공통 키워드 개수: 51


In [103]:
nj_unique_top300 = nj_set_top300.difference(common_keywords)
meaning_words_nj_unique_top300 = meaningwords_nj_set_top300.difference(common_keywords)

print("기사 키워드 리스트 :", nj_unique_top300) # 200, 300
print("형태소 키워드 리스트 :", meaning_words_nj_unique_top300)


기사 키워드 리스트 : {'강하', '따듯하', '섬세하', '흐리', '착하', '앙증맞', '이쁘', '커다랗', '또렷하', '바르', '그렇', '걸맞', '멍하', '푸짐하', '텁텁하', '뿌듯하', '고르', '그러하', '목마르', '색다르', '생소하', '불쌍하', '좋아하', '핫하', '편하', '다르', '답답하', '쓰리', '아프', '뛰어나', '여리', '부시', '소소하', '어리석', '힘차', '폭넓', '충분하', '깜찍하', '따스하', '따뜻하', '이러하', '어리', '간편하', '재밌', '철없', '아무렇', '역하', '노랗', '슬프', '똑같', '궁금하', '쓸데없', '지저분하', '거칠', '푸르', '과하', '깨끗하', '재미없', '예민하', '귀하', '새하얗', '눈부시', '맛있', '빡빡하', '친하', '연하', '풋풋하', '정신없', '바쁘', '하얗', '담백하', '달콤하', '마르', '비릿하', '뻑뻑하', '드세', '화기애애하', '진하', '평범하', '동그랗', '대단하', '짓궂', '치밀하', '무난하', '다름없', '촉촉하', '은은하', '상큼하', '멋있', '맛나', '귀찮', '고고하', '빨갛', '예쁘', '상관없', '쿨하', '기쁘', '묘하', '짠하', '흐릿하', '달달하', '요렇', '엄청나', '멋지', '조그마하', '빵빵하', '신기하', '지나치', '반하', '매콤하', '비슷하', '화려하', '어떻', '밋밋하', '기막히', '까맣', '간략하', '심하', '익숙하', '오지', '따끈하', '둥글', '시원하', '남다르', '대견하', '습하', '능수능란하', '푸르르', '재미있', '확고하', '낯설', '빠르', '흔하', '뒤늦', '신선하', '힘들', '약하', '저렇', '알차', '파랗', '자그마하', '괜찮', '저렴하', '냉하', '수많', '알맞', '세세하', '간단하', '이렇', '활기차',

### 긍부정 지수

In [48]:
!pip install transformers


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 14.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 22.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 34.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 45.6 MB/s eta 0:00:00


In [49]:
from transformers import BertTokenizer, BertForSequenceClassification
import torch
import pandas as pd
import torch.nn.functional as F

# KoBERT 모델과 토크나이저 불러오기
tokenizer = BertTokenizer.from_pretrained("monologg/kobert")
model = BertForSequenceClassification.from_pretrained("monologg/kobert")

# 감성 분석 함수 정의
def sentiment_analysis(text):
    input_ids = tokenizer.encode(text, add_special_tokens=True, return_tensors="pt")
    outputs = model(input_ids)[0]
    probabilities = F.softmax(outputs, dim=1)[0]
    positive_probability = probabilities[1].item() * 100
    return positive_probability

# 예시 키워드 리스트
keywords = ["행복", "슬픔", "즐거움", "우울함", "사랑", "짜증"]

# 키워드 감성 점수 파악
keyword_probabilities = {}
for keyword in keywords:
    probability = sentiment_analysis(keyword)
    keyword_probabilities[keyword] = probability

# 결과 출력
print(keyword_probabilities)


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at monologg/kobert and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


{'행복': 49.39699172973633, '슬픔': 49.63257908821106, '즐거움': 49.63257908821106, '우울함': 49.63257908821106, '사랑': 46.925416588783264, '짜증': 49.63257908821106}


In [50]:
def restore_sentences(text, keywords):
    # sentences = text.split('. ')  # 문장 단위로 분리
    selected_sentences = []
    for sentence in text:
        if any(keyword in sentence for keyword in keywords):  # 키워드가 들어간 문장을 찾음
            selected_sentences.append(sentence)
    return '. '.join(selected_sentences)  # 선택된 문장들을 복원하여 하나의 텍스트로 반환


In [51]:
restore_sentences(nj_text,'예쁜')

'강남역 헤어샵 차홍룸 강남대로점  매직으로 뉴진스 머리해보기!강남역 헤어샵 차홍룸 강남대로점 반곱슬 머리를 매직으로, 뉴진스 머리해보기! 폭풍같던 6  7월이 지나고, 머리를 언제 했는지 기억도 안나는데 장마철도 다 지나고서야 매직을 하기로 했다.원래 6월에 했어야 하는데 정말정말 시간이 없었다 흑... 차홍룸 강남대로점강남역 신분당선 가까이 위치, 서초구 강남대로 337 입구에는 가지런히 놓여져 있는 차홍룸의 안주인? 차홍님께서 쓰신 간단한 에세이가 보였다. 차홍룸은 처음이 아니라서 우드  화이트의 인테리어가 새삼 새롭지는 않았지만 확실히 여느 헤어샵이랑은 다르게 고급스럽고 따듯한 느낌이 난다. 요기 보이는 것은 특정 시술을 했을때 주시는 헤어제품 샘플 같았다 ?? 매직을 할 예정이었기 때문에 머리는 따로 안말았다.겉에만 대충 고데기로 펴고 갔는데 뒤쪽엔 확실히 부스스한게 잘 보인다. 정말 여름 내내 머리때매 너무 고생했는데 진작 매직을 했어야 하는데...요즘 매직한 머리가 뉴진스 때문에 유행이라던데 그냥 고데기도 하지 말까 싶기도 하다.......나이드니까 넘 귀차나 일요일을 길게 쓰려고 오전 첫 타임으로 예약을 했더니 이렇게 여유로운 매장을 찍을 수 있었다. 그래도 사람이 30분만에 가득 차버렸지만 더운 날에도 시원한 공간에서 햇살을 받으니 기분이 좋아진다. 확실히 머리가 부스스 해보이고, 위에는 고데기를 했지만 머리숱이 새삼 많아 보인다 ㅎㅎ 햇살이 좋아서 차홍룸 인증샷 하나 찍어주고 ㅎㅎㅎ 차홍룸은 꼭 거울 앞이 아니라이렇게 창문 앞이나 이렇게 탁자같은 곳에서 시술을 받을 수 있게 해 주신다. 덕분에 책도 더 편하게 읽고 닫혀 있는 공간에서 나만 있으니까 거울 앞에 내내 있는 것 보다 안정감이 느껴지는 것 같다. 머리가 많이 상해서 끝부분을 살짝 자른다고 배정받았던 자리에서 앞쪽에 놓여 있던 제품들이다. 햇볕을 받아서 색감도 예쁘고 동일한 느낌이 좋아서 진 한 컷 찍어보았다.나중에 최중연디자이너 님께서 여기 제품으로 헤어 에센스 마무리를 해 주시기도

### 키워드별 유사도

In [52]:
[진지함(Sincerity):

성실한
신뢰할 수 있는
솔직한
정직한
깊은
참된
진실한
진정성있는
솔직무별한
진심을전하는
책임감있는
진지무진한
어른스러운
겸손한
자긍심없는
온화한
성심성의껏
진심어린
국물하지않은
속죄하는
고운
차분한
실속있는
따뜻한
감히
불가사의한
침착한
빈정대는
건강한
단호한
근면한
자랑스러운
수수한
느긋한
책임감있는
믿음직한
깊은민정
진심어린
자연스러운
고요한
열정(Excitement):

열정적인
활기찬
흥분한
패기있는
감동적인
끓어오르는
다이내믹한
도전적인
불타는
화끈한
열렬한
의욕적인
기쁨찬
열중하는
감격스러운
열성적인
애정어린
강렬한
열광적인
화려한
기쁘고
패기있게
신이난
판타스틱한
몰입하는
흥분높은
흥미진진한
의지있는
들뜬
열렬무렬한
의욕왕성한
이루고싶은
황홀한
끓어오르게
환희에넘치는
열정어린
감격스러운
황홀한
열심인
기운찬
강렬함(Ruggedness):

강렬한
단호한
단호한
화려한
견고한
무자비한
대담한
건장한
집요한
강인한
더운
과감한
질긴
무자비한
경쾌한
탐닉스러운
날카로운
충만한
노골적인
무시무시한
건장한
두꺼운
뜨거운
빈틈없는
건장한
단호한
근엄한
견고한
우뚝세운
통쾌한
어두운
엄격한
역동적인
정책적인
따뜻한
튼튼한
열혈인
엄격한
산뜻한
화려한
겸손함(Humility):

겸손한
수수한
국물하지않은
자부심없는
가식적이지않은
허영심이없는
자만하지않은
과시하지않는
겸손미풍한
소박한
낮은
허세를부리지않는
빼어난
삼가는
천진난만한
단념하는
신사적인
마음속깊은
겸손한
겸손미풍한
선량한
허영심이없는
국물하지않은
이정범하지않은
겸손미풍한
겸손한
자만하지않은
가장저인
산뜻한
소소한
자부심없는
수줍어하는
허심탄회한
낮은
경박한
다정한
겸손미풍한
소박한
실속있는]

SyntaxError: ignored

### 키워드별 이미지 / 색상 등